#Based on Aurélien Géron Template
# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook EXO_TD_1_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia PyPlot"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

# Listing des paquets disponibles dans l'environnement actuel
Si la librairie LibFEM est absente on peut utiliser les 2 commandes suivantes indiféremmen using Pkg; Pkg.add(PackageSpec(url="https://github.com/amdeld/LibFEM.jl")) ou ]add https://github.com/amdeld/LibFEM.jl

In [ ]:
using Pkg; Pkg.add(PackageSpec(url="https://github.com/amdeld/LibFEM.jl"))

# On charge ensuite la librairie de calcul LibFEM et la librairie graphique PyPlot

In [ ]:
using LibFEM, PyPlot

# Quelques informations quant au cas à traiter

In [ ]:
#PARAMETERS
const L=10000. #length in mm
const A=100. #cross-sectional area in mm^2
const E=210000. #modulus of elasticity in MPa [steel]
const FM=10000.; #force modulus in N

In [ ]:
# ===============================================PRE-PROCESSING==================
# DEFINING AND DISCRETIZING[MESHING] THE STRUCTURE
# connectivity table
# elt||node_i||node_j
# 1|1|3
# 2|2|3
# grid()
X1pos=0.;Y1pos=0.
X2pos=0.;Y2pos=L
X3pos=L;Y3pos=L
#lengths
L1=d2_truss_elementlength(X1pos,Y1pos,X3pos,Y3pos) #length of element 1
L2=d2_truss_elementlength(X2pos,Y2pos,X3pos,Y3pos) #length of element 2
#APPLYING GEOMETRIC&MATERIAL PROPERTIES
A1=sqrt(2)*A; #cross-sectional area of element 1
A2=A; #cross-sectional area of element 2
E1=E; #material of element 1
E2=E; #material of element 2

In [ ]:
#writing-defining the element stiffness matrices
K1=d2_truss_elementstiffness(E1,A1,L1,45);
println("K1=\r")
K1

In [ ]:
K2=d2_truss_elementstiffness(E2,A2,L2,0);
println("K2=\r")
K2

## Affichage des matrices de raideurs positionnées $[K_1]^p$ $[K_2]^p$ et de la matrice d'assemblage $[K]$

In [ ]:
#ASSEMBLING THE GLOBAL STIFFNESS MATRIX
#matrices initialization
K=zeros(6,6);K1P=zeros(6,6);
#positionning stiffness matrices
K1P=d2_truss_assemble(K,K1,1,3)
println("K1P=\r")
K1P

In [ ]:
K=zeros(6,6);K2P=zeros(6,6);
K2P=d2_truss_assemble(K,K2,2,3)
println("K2P=\r")
K2P

In [ ]:
#assembling
K=K1P+K2P
println("K=\r")
K

# Affichage des vecteurs des déplacements et des chargements nodaux

In [ ]:
#SOLVING DISPACEMENT EQUATIONS
#extracting displacement submatrix via index vector
K_s=K[5:6,5:6]
#Setting-up the force subvector by applying Load & Boundary Conditions[LBC]]
F_s=[0, -FM]
#solving by gaussian elimination
U_s=K_s\F_s
#SOLVING FORCE EQUATIONS
#setting-up the global nodal displacement vector
U=[0, 0, 0, 0, U_s[1], U_s[2]]
println("U=\r")
U

In [ ]:
#computing the global nodal force vector
F=K*U
println("F=\r")
F

# Calcul et Affichage des grandeurs dérivées déformations et contraintes élémentaires

In [ ]:
#writing the element nodal displacement vectors
U1=[U[1], U[2], U[5], U[6]]
U2=[U[3], U[4], U[5], U[6]]
#computing element strains
ϵ1=d2_truss_elementstrain(L1,45,U1)
@show ϵ1
ϵ2=d2_truss_elementstrain(L2,0,U2)
@show ϵ2;

In [ ]:
#computing element forces
f1=d2_truss_elementforce(E1,A1,L1,45,U1)
@show f1
f2=d2_truss_elementforce(E2,A2,L2,0,U2)
@show f2;

In [ ]:
#computing element stresses
σ1=d2_truss_elementstress(E1,L1,45,U1)
@show σ1
σ2=d2_truss_elementstress(E2,L2,0,U2)
@show σ2;

# Plotting

In [ ]:
Xini=[X1pos, X3pos]
Yini=[Y1pos, Y3pos]
plot(Xini,Yini,"ks-")
Xini=[X2pos, X3pos]
Yini=[Y2pos, Y3pos]
plot(Xini,Yini,"ks-")
fampl=100
Xdef=[(X1pos+fampl*U[1]), (X3pos+fampl*U[5])]
Ydef=[(Y1pos+fampl*U[1]), (Y3pos+fampl*U[6])]
plot(Xdef,Ydef,"bs-")
Xdef=[(X2pos+fampl*U[3]), (X3pos+fampl*U[5])]
Ydef=[(Y2pos+fampl*U[4]), (Y3pos+fampl*U[6])]
plot(Xdef,Ydef,"bs-")